# Progressive changes to the Liberal Democrats UBI Working Group's reforms

Yesterday, we released an [analysis](https://www.ubicenter.org/lib-dem-policy-paper) of universal basic income reforms described in the Liberal Democrat UBI Working Group's [discussion paper](https://d3n8a8pro7vhmx.cloudfront.net/libdems/pages/1811/attachments/original/1621669347/145_-_Universal_Basic_Income.docx_%281%29.pdf?1621669347). In the working group's words, their paper "is designed to stimulate debate and discussion within the Party and outside; based on the response generated and on the deliberations of the working group a full policy paper will be drawn up and presented to Conference for debate."

In this follow-up, we model a budget-neutral version of the reforms put forth in the working group paper, as well as a variety of (also budget-neutral) adjustments to those reforms. While the working group's reform would reduce poverty and inequality, we find that these adjustments, which make the UBIs more generous and inclusive, would augment their effects substantially.

## The working group's UBI reform

The working group paper suggested a range of UBI amounts: 45, 60, 75, and 95 pounds per week for working-age adults. The cost was partly offset by lowering the annual Personal Allowance (for working-age adults) from £12,500 to £2,500 and the weekly National Insurance Primary Threshold from £184 to £50[^1]; these both reduce the amount of earnings one can receive before being taxed. The reform also treated the UBI as earned income for the purposes of means-tested benefits.

The result was progressive: poverty would fall by at least 17 percent, deep poverty (the population share with income below half the poverty threshold) by at least 50 percent, and inequality (the Gini index) by at least 2.8 percent. However, even the least generous of the four UBI policies added over £20 billion to the deficit.

If made budget-neutral, this structure of pay-fors will fund a UBI of £x per week. That policy would reduce poverty by x, deep poverty by x, and inequality by x, while leaving x% of Britons better off and y% worse off. For an apples-to-apples comparison, we consider this the baseline reform against which other adjustments are evaluated.

## Adjustments to the working group's UBI reform

We apply several modifications to the working group's UBI reform that broaden the tax base (funding larger UBIs), broaden the recipient base, and change how the UBI is treated. Specifically, we model:
- Including children in the UBI
- Repealing the Personal Allowance and National Insurance Primary Threshold entirely
- Including pensioners in both the Personal Allowance reduction and the UBI
- Exempting the UBI from means-testing

In addition to modeling each of these changes individually, we consider applying all four changes together.

[^1]: A higher PA (£4,000) and PT (£50) was used for the first reform.

In [12]:
from ubicenter import format_fig
from openfisca_uk import Microsimulation
import numpy as np
import pandas as pd
import plotly.express as px
from reform import (
    WA_adult_UBI,
    all_UBI,
    adult_UBI,
    non_pensioner_UBI,
    set_PA,
    set_PT,
    set_PA_for_WA_adults,
    include_UBI_in_means_tests,
    net_cost,
)

reform_df = pd.DataFrame(
    {
        "Adult PA (£/year)": [12500, 2500, 0, 2500, 2500, 2500, 0],
        "Pensioner PA (£/year)": [12500, 12500, 12500, 2500, 12500, 12500, 0],
        "NI Primary Threshold (£/week)": [183, 50, 0, 50, 50, 50, 0],
        "UBI for children": [False, False, False, False, True, False, True],
        "UBI for pensioners": [False, False, False, True, False, False, True],
        "UBI in means tests": [False, True, True, True, True, False, False],
    }
)

baseline = Microsimulation(year=2020)


def create_reform(params: dict):
    reform = []
    reform += [set_PA(float(params["Pensioner PA (£/year)"]))]
    reform += [set_PA_for_WA_adults(float(params["Adult PA (£/year)"]))]
    reform += [set_PT(float(params["NI Primary Threshold (£/week)"]))]
    tax_reform_sim = Microsimulation(*reform, year=2020)
    revenue = net_cost(tax_reform_sim, baseline)
    if params["UBI for children"]:  # doesn't handle non-adult UBIs
        if params["UBI for pensioners"]:
            ubi_reform_func = all_UBI
            population = baseline.calc("people").sum()
        else:
            ubi_reform_func = non_pensioner_UBI
            population = (
                baseline.calc("is_child").sum()
                + baseline.calc("is_WA_adult").sum()
            )
    else:
        if params["UBI for pensioners"]:
            ubi_reform_func = adult_UBI
            population = baseline.calc("is_adult").sum()
        else:
            ubi_reform_func = WA_adult_UBI
            population = baseline.calc("is_WA_adult").sum()
    if params["UBI in means tests"]:
        ubi_amount = int(revenue / population / 52) * 52
        net_revenue = -net_cost(
            baseline,
            Microsimulation(
                (
                    reform,
                    ubi_reform_func(ubi_amount),
                    include_UBI_in_means_tests(),
                ),
                year=2020,
            ),
        )
        prev_amounts = []
        while (
            net_revenue > 1e9 or net_revenue < -1e9
        ) and ubi_amount not in prev_amounts:
            old_ubi_amount = ubi_amount
            prev_amounts += [old_ubi_amount]
            ubi_amount += 1 * 52 * (2 * (net_revenue > 0) - 1)
            net_revenue = -net_cost(
                baseline,
                Microsimulation(
                    (
                        reform,
                        ubi_reform_func(ubi_amount),
                        include_UBI_in_means_tests(),
                    ),
                    year=2020,
                ),
            )
        reform += [ubi_reform_func(ubi_amount), include_UBI_in_means_tests()]
    else:
        ubi_amount = int(revenue / population / 52) * 52
        reform += [ubi_reform_func(ubi_amount)]
    return tuple(reform)


def rel(x, y):
    return (y - x) / x


UBI_amounts = []
poverty_changes = []
deep_poverty_changes = []
costs = []
winners = []
losers = []
gini_changes = []

from tqdm import trange

for i in range(len(reform_df)):
    reform = create_reform(reform_df.iloc[i])
    reform_sim = Microsimulation(reform, year=2020)
    UBI_amounts += [reform_sim.calc("UBI").max()]
    poverty_changes += [
        rel(
            baseline.calc("in_poverty_bhc", map_to="person").mean(),
            reform_sim.calc("in_poverty_bhc", map_to="person").mean(),
        )
    ]
    deep_poverty_changes += [
        rel(
            baseline.calc("in_deep_poverty_bhc", map_to="person").mean(),
            reform_sim.calc("in_deep_poverty_bhc", map_to="person").mean(),
        )
    ]
    gini_changes += [
        rel(
            baseline.calc("household_net_income", map_to="person").gini(),
            reform_sim.calc("household_net_income", map_to="person").gini(),
        )
    ]
    winners += [
        (
            reform_sim.calc("household_net_income", map_to="person")
            > baseline.calc("household_net_income", map_to="person") + 1
        ).mean()
    ]
    losers += [
        (
            reform_sim.calc("household_net_income", map_to="person")
            < baseline.calc("household_net_income", map_to="person") - 1
        ).mean()
    ]
    costs += [net_cost(baseline, reform_sim)]
    
results_df = pd.DataFrame(
    {
        "UBI amount (£/week)": (pd.Series(UBI_amounts) / 52).astype(int),
        "Poverty change (%)": pd.Series(poverty_changes).apply(
            lambda x: round(x * 100, 1)
        ),
        "Deep poverty change (%)": pd.Series(deep_poverty_changes).apply(
            lambda x: round(x * 100, 1)
        ),
        "Winners (%)": pd.Series(winners).apply(lambda x: round(x * 100, 1)),
        "Losers (%)": pd.Series(losers).apply(lambda x: round(x * 100, 1)),
        "Inequality change (%)": pd.Series(gini_changes).apply(
            lambda x: round(x * 100, 1)
        ),
        "Net cost (£bn/year)": pd.Series(costs).apply(
            lambda x: round(x / 1e9, 1)
        ),
    }
)

output = pd.concat([reform_df, results_df], axis=1)
output.index = [
    "Baseline",
    "Budget-neutral Working Group reform",
    "Full PA/PT elimination",
    "Include pensioners",
    "Include children",
    "Exclude from means tests",
    "All",
]
output


,Adult PA (£/year),Pensioner PA (£/year),NI Primary Threshold (£/week),UBI for children,UBI for pensioners,UBI in means tests,UBI amount (£/week),Poverty change (%),Deep poverty change (%),Winners (%),Losers (%),Inequality change (%),Net cost (£bn/year)
Baseline,12500,12500,183,False,False,False,0,0.0,0.0,0.0,0.0,0.0,0.0
Budget-neutral Working Group reform,2500,12500,50,False,False,True,42,-10.9,-46.0,38.9,47.1,-2.4,-0.2
Full PA/PT elimination,0,12500,0,False,False,True,55,-12.9,-56.3,38.1,48.1,-2.9,-0.7
Include pensioners,2500,2500,50,False,True,True,39,-16.8,-45.1,49.9,49.2,-3.2,0.8
Include children,2500,12500,50,True,False,True,32,-21.1,-49.6,47.6,38.8,-3.3,-0.1
Exclude from means tests,2500,12500,50,False,False,False,37,-17.3,-43.7,41.4,45.2,-3.1,-1.5
All,0,0,0,True,True,False,36,-39.6,-54.4,52.6,47.1,-5.5,-1.7
